In [ ]:
!pip install -q google-genai tqdm

# %%
from google import genai
from google.genai import types
from pathlib import Path
import os, json, time, re, shutil, tqdm
from typing import List, Dict


In [ ]:
# ## 1  Authenticate & Drive

# %%
os.environ['GEMINI_API_KEY'] = ''
client = genai.Client()
print('Gemini client ready')

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

drive_root = Path('/content/drive/MyDrive/KGP_evidence')
work_dir   = Path('/content/llm_answer_generation')
work_dir.mkdir(parents=True, exist_ok=True)
for filename in ['hotpotqa_evidence_qwen.json', '2wiki_evidence_qwen.json']:
    shutil.copy2(drive_root/filename, work_dir/filename)
print('📄 Evidence copied')

✅ Gemini client ready
Mounted at /content/drive
📄 Evidence copied


In [ ]:
# ## 2  Helpers

# %%

def load_json(path: Path) -> List[Dict]:
    with open(path, 'r') as f:
        return json.load(f)


def save_json(obj, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w') as f:
        json.dump(obj, f, indent=4, ensure_ascii=False)


# — safe clean
def _clean_answer(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"<think[^>]*?>.*?</think>", " ", text, flags=re.S)
    return re.sub(r"\s+", " ", text).strip()


def split_thoughts_and_answer(parts):
    thoughts, answer = [], []
    for p in parts:
        txt = getattr(p, 'text', '') or ''
        if getattr(p, 'thought', False):
            thoughts.append(txt)
        else:
            answer.append(txt)
    return "\n".join(thoughts).strip(), " ".join(answer).strip()


In [ ]:
# ## 3  Generation pipeline (robust)

# %%

def generate_answers(
    evidence_file: Path,
    save_file: Path,
    *,
    max_output_tokens: int = 128,
    temperature: float = 0.6,
    top_p: float = 0.95,
    thinking_budget: int = -1,
    pause_secs: float = 5.0,
):
    data = load_json(evidence_file)
    responses = []

    prompt_template = (
        "Given the question and its associated contexts below, please generate a concise, "
        "precise answer in English. The answer must strictly adhere to the following guidelines:\n"
        "- The answer should be directly relevant to the question.\n"
        "- Provide the answer in a clear, straightforward format.\n"
        "- Limit your answer to no more than 10 words, focusing on the essential information requested.\n"
        "- If the provided contexts do not contain enough information but you know the answer, still provide it; "
        "otherwise respond with \"Information not available\".\n"
        "- Do not include any additional tokens, explanations, or information beyond the direct answer.\n\n"
        "QUESTION: {question}\n"
        "CONTEXT:\n{context}\n"
        "ANSWER:"
    )

    none_template = (
        "Given the following question, create a final answer in English to the question.\n"
        "QUESTION: {question}\n"
        "ANSWER:"
    )

    for rec in tqdm.tqdm(data, total=len(data)):
        ctx_list = rec['evidence']
        user_prompt = (prompt_template if ctx_list else none_template).format(
            question=rec['question'],
            context="\n".join(f"{i+1}: {c}" for i, c in enumerate(ctx_list)) if ctx_list else ""
        )

        try:
            response = client.models.generate_content(
                model='gemini-2.5-flash',
                contents=user_prompt,
                config=types.GenerateContentConfig(
                    temperature=temperature,
                    top_p=top_p,
                    max_output_tokens=max_output_tokens,
                    thinking_config=types.ThinkingConfig(
                        thinking_budget=thinking_budget,
                        include_thoughts=True
                    ),
                ),
            )

            cand_container = response.candidates or []
            cand = cand_container[0] if cand_container else None
            parts = cand.content.parts if cand else []
            thoughts, answer_text = split_thoughts_and_answer(parts)
            short_answer = _clean_answer(answer_text) or "Information not available"

        except Exception as e:
            thoughts = ""
            answer_text = f"ERROR: {e}"
            short_answer = "Information not available"

        responses.append({
            'type':     rec['type'],
            'question': rec['question'],
            'gt':       rec['answer'],
            'thoughts': thoughts,
            'answer':   answer_text,
            'response': short_answer,
        })

        save_json(responses, save_file)
        time.sleep(pause_secs)

    print(f"Finished – saved to {save_file}")
    return responses

In [ ]:
# %%
wiki_out = drive_root / '2wiki_answers_gemini.json'
_ = generate_answers(
        work_dir / '2wiki_evidence_qwen.json',
        wiki_out,
        max_output_tokens=1024,
        temperature=0.6,
        top_p=0.95,
        thinking_budget=-1,
        pause_secs=6.0,
)


100%|██████████| 100/100 [14:58<00:00,  8.99s/it]

Finished – saved to /content/drive/MyDrive/KGP_evidence/2wiki_answers_gemini.json
